Библиотека Pandas — это весьма эффективный инструмент для обработки данных, представляющих собой временные ряды. На самом деле, эта библиотека была создана Уэсом МакКинни для работы с финансовыми данными, которые состоят, главным образом, из временных рядов.

При работе с временными рядами много сил уходит на выполнение различных операций с датой и временем. Этот материал посвящён ответам на четыре распространённых вопроса из данной сферы.

Возможно, вы уже сталкивались с этими вопросами. Ответить на все из них, кроме последнего, можно сравнительно просто. А вот ответ на последний, довольно-таки хитрый вопрос, представляет собой последовательность из нескольких действий.

Начнём с создания учебного датафрейма (объекта DataFrame), с которым будем экспериментировать:

In [2]:
import pandas as pd

In [4]:
df = pd.DataFrame({
    "booking_id": [1001, 1002, 1003, 1004, 1005],
    "property" : ["A", "A", "B", "B", "C"],
    "created_at": ["2022-03-01", "2022-02-10", "2022-04-12",
                   "2022-04-11", "2022-06-05"],
    "checkin_date": ["2022-06-01", "2022-06-10", "2022-06-02",
                     "2022-06-20", "2022-08-10"],
    "checkout_date": ["2022-06-06", "2022-06-15", 
                      "2022-06-06","2022-06-28", "2022-08-16"],
    "amount": [5400, 5600, 4800, 9000, 6500]
})
# изменение типа данных 
date_cols = ["created_at","checkin_date","checkout_date"]
df[date_cols] = df[date_cols].astype("datetime64[ns]")
# вывод объекта DataFrame
df

,booking_id,property,created_at,checkin_date,checkout_date,amount
0,1001,A,2022-03-01,2022-06-01,2022-06-06,5400
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500


Для того чтобы у нас была бы возможность пользоваться функциями pandas, предназначенными для работы с датой и временем, даты в датафрейме должны быть представлены значениями подходящего типа данных. Именно по этой причине мы поменяли тип данных соответствующих столбцов на datetime64[ns].

1. Как извлечь из даты информацию о месяце и годе?
Дата содержит различные фрагменты информации, такие как год, день недели, месяц и так далее. Все эти фрагменты можно извлечь из даты с использованием методов, к которым обращаются через аксессор dt.

Например, месяц можно получить с помощью метода month. А как получить сведения о годе и месяце в формате ГГГГ-ММ? Ответ на этот вопрос уже не так очевиден. Извлечь эти сведения из даты можно, прибегнув к методу to_period:

In [8]:
# создание столбца year_month
df["year_month"] = df["created_at"].dt.to_period("M")
# вывод объекта DataFrame
df

,booking_id,property,created_at,checkin_date,checkout_date,amount,year_month
0,1001,A,2022-03-01,2022-06-01,2022-06-06,5400,2022-03
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600,2022-02
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800,2022-04
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000,2022-04
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500,2022-06


Как прибавить к дате временной интервал?
Прибавление к датам временных интервалов или вычитание их из дат — это одна из операций, часто встречающихся при работе с датами. Выполнить её можно, воспользовавшись функцией DateOffset.

Добавим 1 день к дате выселения (например — из некоего объекта недвижимости) (checkout_date) для записи о бронировании с идентификатором 1001:

In [11]:
df.loc[df["booking_id"]==1001, "checkout_date"] = \
df.loc[df["booking_id"]==1001, "checkout_date"] + \
pd.DateOffset(days=1)
# проверка результата
print(df.loc[df["booking_id"]==1001, "checkout_date"])
# вывод 0   2022-06-07
# Name: checkout_date, dtype: datetime64[ns]

0   2022-06-08
Name: checkout_date, dtype: datetime64[ns]


3. Как найти интервал в днях между двумя датами?
Интервал между двумя датами можно найти, вычтя одну из другой. Результатом этой операции будет объект Timedelta, а это не совсем то, что нам нужно:

In [13]:
df["checkout_date"][0] - df["checkin_date"][0]
# вывод Timedelta('6 days 00:00:00')

Timedelta('7 days 00:00:00')

Получить разницу между датами в днях, в виде целого числа, можно, воспользовавшись атрибутом days. Создадим столбец, который выводит разницу в днях между датой вселения (checkin_date) и датой создания бронирования (created_at):

In [14]:
# разница, выраженная в днях
df["days_to_checkin"] = \
(df["checkin_date"] - df["created_at"]).dt.days
# вывод объекта DataFrame
df

,booking_id,property,created_at,checkin_date,checkout_date,amount,year_month,days_to_checkin
0,1001,A,2022-03-01,2022-06-01,2022-06-08,5400,2022-03,92
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600,2022-02,120
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800,2022-04,51
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000,2022-04,70
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500,2022-06,66
